<a href="https://colab.research.google.com/github/AnIsAsPe/Estadistica_y_Probabilidad_para-CD-/blob/main/PrincipiosDeProbabilidad/Semana1/Notebooks/Distribucion_multinomial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Instalar y cargar librerias

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
!python -m spacy download es_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 51.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [14]:
import numpy as np
import pandas as pd
import spacy
nlp = spacy.load("es_core_news_sm")
import re

import matplotlib.pyplot as plt

## Definición de funciones

In [2]:
def leer_libro(archivo_txt):
    '''funcion para leer un archivo en formato txt'''
    with open(archivo_txt, "r", encoding="utf8") as f:
      texto = f.read()
      texto = texto.replace("\n", "").replace("\r", "").replace("«", "").replace("»", "").replace("-", "")
    return texto

In [3]:
def borrar_elementos(texto, lista):
    '''Función para eliminar en un texto los elementos contenidos en una lista,
    por ejemplo:   autor, titulo y otros elementos'''

    for elemento in lista:
      texto = texto.replace(elemento, "")
    return texto

In [51]:
def frases_de_longitud_d(texto:str, d:int):
    df = pd.DataFrame(texto.split('. '), columns=['Frases'])
    df['Frases']=df['Frases'].apply(lambda x:  " ".join(x.split()[:d]))
    df['TotPalabras'] = df['Frases'].apply(lambda x: len(x.split()))
    df_d = df[df['TotPalabras'] >= d].reset_index(drop=True).copy()
    return df_d

In [52]:
def variables_multinomiales(texto:str):
    texto_nlp = nlp(texto)
    pos_tags = [token.pos_ for token in texto_nlp][:n]
    lista = ['NOUN', 'PROPN', 'VERB', 'PUNCT']
    pos_tags_modified = ['OTRO' if x not in lista else x for x in pos_tags]
    assert len(pos_tags_modified) == n, f"La longitud de {texto} es {len(pos_tags_modified)}"
    return pos_tags_modified

In [61]:
def cuenta_palabras(texto:str):
    texto_nlp = nlp(texto)
    pos_tags = [token.pos_ for token in texto_nlp][:n]
    nombres = len([token for token in pos_tags if token in 'NOUN'])
    nombres_propios = len([token for token in pos_tags if token in 'PROPN'])
    verbos = len([token for token in pos_tags if token == 'VERB'])
    puntuacion = len([token for token in pos_tags if token == 'PUNCT'])
    otras = len(pos_tags) - nombres - nombres_propios - verbos - puntuacion
    frecuencia = {'nombre': nombres, 'nombrepropio': nombres_propios, 'verbo': verbos, 'puntuacion': puntuacion, 'otra': otras}
    return frecuencia

## Leer y prepocesar texto

In [7]:
libro = leer_libro('/content/drive/MyDrive/Datos/gabriel_garcia_marquez_cien_annos_soledad.txt')
borrar = ['Gabriel García Márquez',
          'Cien años de soledad',
          'EDITADO POR "EDICIONES LA CUEVA"',
          'Para J omi García Ascot y María Luisa Elio']
libro =  borrar_elementos(libro, borrar)
libro = " ".join(libro.split())


In [62]:
n = 17
df = frases_de_longitud_d(libro, n)
df[[i for i in range (1,n+1)]] = df['Frases'].apply(variables_multinomiales).apply(pd.Series)
df[['NOUN', 'PROPN', 'VERB', 'PUNCT','OTRO']] = df['Frases'].apply(cuenta_palabras).apply(pd.Series)
df['Total'] = df[['NOUN', 'PROPN', 'VERB', 'PUNCT','OTRO']].sum(axis=1)
df

,Frases,TotPalabras,1,2,3,4,5,6,7,8,...,14,15,16,17,NOUN,PROPN,VERB,PUNCT,OTRO,Total
0,"Muchos años después, frente al pelotón de fusi...",17,OTRO,NOUN,OTRO,PUNCT,NOUN,OTRO,NOUN,OTRO,...,PROPN,OTRO,OTRO,VERB,5,2,1,2,7,17
1,Macondo era entonces una aldea de veinte casas...,17,PROPN,OTRO,OTRO,OTRO,NOUN,OTRO,OTRO,NOUN,...,OTRO,OTRO,NOUN,OTRO,4,1,0,0,12,17
2,"El mundo era tan reciente, que muchas cosas ca...",17,OTRO,NOUN,OTRO,OTRO,OTRO,PUNCT,OTRO,OTRO,...,OTRO,OTRO,NOUN,OTRO,4,0,1,2,10,17
3,"Todos los años, por el mes de marzo, una famil...",17,OTRO,OTRO,NOUN,PUNCT,OTRO,OTRO,NOUN,OTRO,...,NOUN,OTRO,VERB,OTRO,5,0,1,2,9,17
4,"Un gitano corpulento, de barba montaraz y mano...",17,OTRO,NOUN,OTRO,PUNCT,OTRO,NOUN,OTRO,OTRO,...,OTRO,VERB,OTRO,OTRO,4,0,1,2,10,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3428,"Aureliano lo reconoció, persiguió los caminos ...",17,PROPN,OTRO,VERB,PUNCT,VERB,OTRO,NOUN,OTRO,...,VERB,OTRO,NOUN,OTRO,3,1,3,2,8,17
3429,"Estaba tan absorto, que no sintió tampoco la s...",17,OTRO,OTRO,OTRO,PUNCT,OTRO,OTRO,VERB,OTRO,...,PUNCT,OTRO,NOUN,OTRO,3,0,1,2,11,17
3430,Sólo entonces descubrió que Amaranta Úrsula no...,17,OTRO,OTRO,VERB,OTRO,PROPN,PROPN,OTRO,OTRO,...,NOUN,PUNCT,OTRO,OTRO,2,2,1,2,10,17
3431,Macondo era ya un pavoroso remolino de polvo y...,17,PROPN,OTRO,OTRO,OTRO,OTRO,NOUN,OTRO,NOUN,...,NOUN,OTRO,NOUN,OTRO,5,1,0,0,11,17
